In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Union
from itertools import product, chain

In [3]:
gauges = gpd.read_file("data/geometry/qh_gauges.gpkg")
gauges = gauges.set_index('gauge_id')
gauges

,name_ru,name_en,geometry
gauge_id,,,
19313,р.Худолаз - пос.Чернышевский,r.Hudolaz - pos.Chernyshevskij,POINT (58.82645 52.63939)
75236,р.Касть - д.Рылово,r.Kast' - d.Rylovo,POINT (40.45643 58.01418)
72559,р.Систа - д.Среднее Райково,r.Sista - d.Srednee Rajkovo,POINT (28.8 59.73)
75222,р.Печегда - пос.Чебаково,r.Pechegda - pos.Chebakovo,POINT (39.56053 57.75352)
71044,р.Ура - с.Ура-Губа,r.Ura - s.Ura-Guba,POINT (32.77889 69.26977)
...,...,...,...
77362,р.Большой Караман - пгт Советское,r.Bol'shoj Karaman - pgt Sovetskoe,POINT (46.75 51.45)
3577,р.Чечуй - д.Пущино,r.Chechuj - d.Puschino,POINT (109.33 58.2)
78092,р.Лесной Воронеж - сл.Заворонежская,r.Lesnoj Voronezh - sl.Zavoronezhskaja,POINT (40.53048 52.88205)


In [ ]:
def flood_extent_tiles(topo_p: Union[Path, str], extent_coords: tuple) -> dict:
    """Return the pathes for .tiff files of elevation and flow direction in given folder.

    Args:
    ----
        topo_p (Union[pathlib.Path, str]): Folder with pre-downloaded files
        extent_coords (tuple): tuple with extent max,min latitude and longitude

    Returns:
    -------
        dict: Dictionary with ['elv'] and ['dir'] tags which has files
        corresponding files for gauge of interest

    """
    x_min, y_min, x_max, y_max = extent_coords

    x_min, y_min = list(map(round_down, [x_min, y_min]))
    x_max, y_max = list(map(round_up, [x_max, y_max]))

    tile_boundaries = [
        f"n{lat}e0{lon}" if len(str(lon)) < 3 else f"n{lat}e{lon}"
        for lat, lon in product(
            np.arange(start=y_min, stop=y_max + 5, step=5),
            np.arange(start=x_min, stop=x_max + 5, step=5),
        )
    ]
    variables = ["dir", "elv"]

    tiles = {
        var: list(
            chain.from_iterable(
                [
                    glob.glob(f"{topo_p}/{var}/**/{boundary}_{var}.tif", recursive=True)
                    for boundary in tile_boundaries
                ]
            )
        )
        for var in variables
    }

    return tiles
